In [ ]:
import os

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

detections_file = "../datasets/experiments/parkeren/detections/combined_detections_centrum.geojson"
buurten_file = "../datasets/experiments/parkeren/bgt/Gebieden/GBD_buurt.shp"

bgt_folder = "../datasets/experiments/parkeren/bgt/Amsterdam_Centrum/"
bgt_voetpad_functions = ['voetpad', 'fietspad', 'voetpad op trap', 'voetgangersgebied']

parkeervlakken_file = "../datasets/experiments/parkeren/bgt//parkeervakken-parkeervakken-2025-09-12T15_32_00.255804+02_00.json"

RD_crs = "EPSG:28992"

car_dimensions = {
    "min_width": 1.4,
    "min_length": 3.0,
    "max_width": 3.1,
    "max_length": 12.0
}

conf_threshold = 0.04

In [ ]:
def get_shape(geom):
    coords = geom.exterior.coords

    # get length of bounding box edges
    edge_length = (
        Point(coords[0]).distance(Point(coords[1])),
        Point(coords[1]).distance(Point(coords[2]))
    )

    return {
        "length": max(edge_length),
        "width": min(edge_length)
    }

def is_valid_detection(row):
    accept_conf = row["confidence"] >= conf_threshold
    accept_width = (
        (row["width"] >= car_dimensions["min_width"])
        and (row["width"] <= car_dimensions["max_width"])
    )
    accept_length = (
        (row["length"] >= car_dimensions["min_length"])
        and (row["length"] <= car_dimensions["max_length"])
    )
    return accept_conf and accept_width and accept_length

In [ ]:
# Load buurten shapes
buurten_gdf = gpd.read_file(buurten_file)
buurten_gdf = buurten_gdf[buurten_gdf["sdl_naam"] == "Centrum"]

# Load BGT wegdeel
_bgt_wgl_gdfs = []
_bgt_wgl_files = [file for file in os.listdir(bgt_folder) if file.startswith("BGT_WGL") and file.endswith(".shp")]
for file in _bgt_wgl_files:
    _bgt_wgl_gdfs.append(gpd.read_file(os.path.join(bgt_folder, file)))

bgt_wgl_gdf = gpd.GeoDataFrame(pd.concat(_bgt_wgl_gdfs))
bgt_wgl_gdf = bgt_wgl_gdf[bgt_wgl_gdf["eindreg"].isna()]

# Filter pedestrian zones
bgt_voetpad_gdf = bgt_wgl_gdf[bgt_wgl_gdf["bgtfunctie"].isin(bgt_voetpad_functions)]

# Load parkeervlakken
parkeervlakken_gdf = gpd.read_file(parkeervlakken_file)
parkeervlakken_gdf.to_crs(RD_crs, inplace=True)
parkeervlakken_gdf = parkeervlakken_gdf[parkeervlakken_gdf.is_valid]
parkeervlakken_gdf = parkeervlakken_gdf[parkeervlakken_gdf.intersects(buurten_gdf.union_all())]

In [ ]:
def get_buurt_and_wijk(row):
    buurt = buurten_gdf[buurten_gdf.contains(row["geometry"].centroid)]

    if len(buurt) > 0:
        buurt = buurt.iloc[0]
        return {
            "buurt_code": buurt["code"], 
            "buurt_naam": buurt["naam"], 
            "wijk_naam": buurt["wijk_naam"]
        }
    
    else:
        return {
            "buurt_code": None, 
            "buurt_naam": None, 
            "wijk_naam": None
        }

In [ ]:
# Load detections from file
centrum_detections_gdf = gpd.read_file(detections_file).set_index("index")

# Enrich detections
centrum_detections_gdf[["length", "width"]] = centrum_detections_gdf.apply(lambda row: get_shape(row["geometry"]), axis="columns", result_type="expand")
centrum_detections_gdf["valid_detection"] = centrum_detections_gdf.apply(is_valid_detection, axis="columns")
centrum_detections_gdf[["buurt_code", "buurt_naam", "wijk_naam"]] = centrum_detections_gdf.apply(get_buurt_and_wijk, axis="columns", result_type="expand")

# Filter valid detections
valid_detections_gdf = centrum_detections_gdf[centrum_detections_gdf["valid_detection"] & ~centrum_detections_gdf["buurt_code"].isna()].copy()

## Compute wrongly parked cars

In [ ]:
voetpad_shape = bgt_voetpad_gdf.union_all() - parkeervlakken_gdf.union_all()

valid_detections_gdf["percentage_on_sidewalk"] = (
    valid_detections_gdf.intersection(voetpad_shape).area 
    / valid_detections_gdf.area
)
valid_detections_gdf["wrongly_parked"] = valid_detections_gdf["percentage_on_sidewalk"] >= 0.25

In [ ]:
print(f"Number of cars: {len(valid_detections_gdf)}")
print(f"Wrongly parked: {valid_detections_gdf['wrongly_parked'].sum()} ({(valid_detections_gdf['wrongly_parked'].sum() / len(valid_detections_gdf)) * 100:.1f}%)")

In [ ]:
valid_detections_gdf["percentage_on_sidewalk"].hist(bins=50)

In [ ]:
valid_detections_gdf[["wijk_naam", "wrongly_parked"]].groupby("wijk_naam").value_counts(normalize=True)